In [263]:
import lithops
import laspy.copc as lasp
import time
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

In [264]:
from cloudAPI import CopcHelper

In [265]:
bucket= 'lithops-testing3'
filekey= 'autzen-classified.copc.laz'
filename= 'lidar.copc.laz'

In [266]:
def download(helper, key):
    helper.load_points(key)

In [267]:
def helper_test(aux, ibm_cos):
    clock0 = time.time()
    
    key = lasp.VoxelKey()
    key.x = 1
    key.y = 2
    key.z = 0
    key.level = 2
    
    #Download header and octree
    helper = CopcHelper(bucket, filekey, filename, ibm_cos)
    helper.load_header()
    helper.load_rootpage()
    
    clock1 = time.time()
    
    #Download and read points of a level 1 node
    helper.load_points(key)
    points = helper.get_points(key)
    
    clock2 = time.time()
    
    #Find neighbours of a level 1 node
    neighbours = helper.get_smallest_neighbours(key)
    
    clock3 = time.time()
    
    helper.multiple_points_download(neighbours)
    
    #Download and load points of neighbours
    neighbour_points = []
    for neighbour in neighbours:
        neighbour_points.append(helper.get_points(neighbour))
    
    clock4 = time.time()

    return clock1 - clock0, clock2 - clock1, clock3 - clock2, clock4 - clock3, key, str(helper.get_level_nodes(2))

In [268]:
if __name__ == "__main__":
    fexec = lithops.LocalhostExecutor()
    fexec.call_async(helper_test, 0)
    results = fexec.get_result()

2022-07-13 12:02:44,280 [INFO] lithops.config -- Lithops v2.6.0
2022-07-13 12:02:44,284 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-07-13 12:02:44,284 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-07-13 12:02:44,285 [INFO] lithops.invokers -- ExecutorID 0b75b8-39 | JobID A000 - Selected Runtime: python 
2022-07-13 12:02:44,319 [INFO] lithops.invokers -- ExecutorID 0b75b8-39 | JobID A000 - Starting function invocation: helper_test() - Total: 1 activations
2022-07-13 12:02:44,353 [INFO] lithops.invokers -- ExecutorID 0b75b8-39 | JobID A000 - View execution logs at /tmp/lithops/logs/0b75b8-39-A000.log
2022-07-13 12:02:45,273 [INFO] lithops.wait -- ExecutorID 0b75b8-39 - Getting results from 1 function activations


    0%|          | 0/1  

2022-07-13 12:02:50,120 [INFO] lithops.executors -- ExecutorID 0b75b8-39 - Cleaning temporary data


In [269]:
    print("EXECUTION TIMES OF KEY ", results[4])
    print("Download header and octree: ", results[0])
    print("Download and read points of a level 2 node: ", results[1])
    print("Find neighbours of a level 2 node: ", results[2])
    print("Download and read points of neighbours: ", results[3])
    print("Total time: ", results[0] + results[1] + results[2] + results[3])

EXECUTION TIMES OF KEY  VoxelKey(level=2, x=1, y=2, z=0)
Download header and octree:  1.0941524505615234
Download and read points of a level 2 node:  0.5279037952423096
Find neighbours of a level 2 node:  0.00512242317199707
Download and read points of neighbours:  2.099402904510498
Total time:  3.726581573486328


In [270]:
    print("All level 2 keys: ", results[5])

All level 2 keys:  [Entry(key=VoxelKey(level=2, x=0, y=0, z=0), offset=77296609, byte_size=425925, point_count=40968, Entry(key=VoxelKey(level=2, x=1, y=0, z=0), offset=78079083, byte_size=527901, point_count=51803, Entry(key=VoxelKey(level=2, x=0, y=1, z=0), offset=78606984, byte_size=383754, point_count=37540, Entry(key=VoxelKey(level=2, x=1, y=1, z=0), offset=78990738, byte_size=471950, point_count=45410, Entry(key=VoxelKey(level=2, x=2, y=0, z=0), offset=76873679, byte_size=422930, point_count=39588, Entry(key=VoxelKey(level=2, x=2, y=1, z=0), offset=77722534, byte_size=356549, point_count=34887, Entry(key=VoxelKey(level=2, x=0, y=2, z=0), offset=67107560, byte_size=298243, point_count=27728, Entry(key=VoxelKey(level=2, x=1, y=2, z=0), offset=68502824, byte_size=246527, point_count=22707, Entry(key=VoxelKey(level=2, x=0, y=3, z=0), offset=70950773, byte_size=438369, point_count=41265, Entry(key=VoxelKey(level=2, x=1, y=3, z=0), offset=72250329, byte_size=322544, point_count=30194, 